**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga1](https://drive.google.com/file/d/1zCIHNAxe5Shc36Qo0XjehHgwrafKSJ_t/view), [liga2](https://drive.google.com/file/d/1RMwUXEN_SOHKue-J9Cx3Ldvj9bejLjiM/view).

In [2]:
!pip3 install --user -q cvxpy

^C
ERROR: Operation cancelled by user


In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/algoritmos/Python'

In [4]:
os.chdir(cur_directory + dir_alg_python)

In [5]:
import math

import numpy as np

from utils_2nd_version import compute_error

from algorithms_for_cieco_2nd_version import path_following_method_infeasible_init_point_2nd_version, \
                                             path_following_method_infeasible_init_point


# Está lista la implementación para puntos iniciales no factibles en $Ax=b$ pero no para la parte de las desigualdades $f_i(x) <0 \quad \forall i=1,\dots,m$

# Primer ejemplo

$$ \min \quad x_1^2 + x_2^2 + x_3^2 + x_4^2 -2x_1-3x_4$$

$$\text{sujeto a: } $$

$$
\begin{array}{c}
2x_1 + x_2 + x_3 + 4x_4 = 7 \\
x_1 + x_2 + 2x_3 + x_4 = 6
\end{array}
$$

$$x_1, x_2, x_3, x_4 \geq 0$$

## Infeasible for Ax=b

In [6]:
from utils_2nd_version import constraint_inequalities_funcs_eval

In [7]:
fo = lambda x: x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2-2*x[0]-3*x[3]

In [8]:
const = {0: lambda x: -x[0],
         1: lambda x: -x[1],
         2: lambda x: -x[2],
         3: lambda x: -x[3]
        }

In [9]:
A= np.array([[2,1,1,4],
             [1,1,2,1]])

In [10]:
b=np.array([7,6])

In [11]:
x_ast=np.array([1.1232876712328763,0.6506849315068493,
                1.8287671232876714,0.5684931506849317])

In [12]:
x_0 = np.array([-5,-5,-5,-5],dtype=float) #with -5,-5,-5,-5 return one entry of x negative

In [13]:
nu_0 = np.array([0,0], dtype=float)

In [14]:
p_ast=fo(x_ast)

In [15]:
p_ast

1.4006849315068495

In [16]:
tol_outer_iter = 1e-6
tol=1e-8
tol_backtracking=1e-8
maxiter=30
mu=10

In [17]:
x = path_following_method_infeasible_init_point_2nd_version(fo, A, b,
                                                                             const,
                                                                             x_0, nu_0, tol,
                                                                             tol_backtracking = tol_backtracking, 
                                                                             x_ast = x_ast, 
                                                                             p_ast=p_ast, 
                                                                             maxiter=maxiter,
                                                                             mu=mu, tol_outer_iter = tol_outer_iter 
                                                                             )

Outer iterations of path following infeasible method
Mu value: 1.00e+01
LogBarrier 	t_log_barrier	 s_infeasible
4.00e+10	3.24e-02	1.50e+01
----------------------------------------------------------------------------------------
gfeval
[-0.48835165 -0.42362637 -0.42362637 -0.52071427  0.        ]
Hfeval
[[0.07484601 0.0004599  0.0004599  0.         0.        ]
 [0.0004599  0.07530591 0.0004599  0.         0.        ]
 [0.0004599  0.0004599  0.07530591 0.         0.        ]
 [0.         0.         0.         0.07484601 0.        ]
 [0.         0.         0.         0.         0.        ]]
I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	5.83e+01	1.03e+02	1.12e+01	8.34e+00	8.82e+01	---		inf
1	1.26e-15	6.75e-01	1.44e-01	9.95e-02	3.78e-02	1.00e+00	inf
2	8.88e-16	6.75e-01	1.44e-01	9.95e-02	3.79e-02	6.10e-05	inf
3	0.00e+00	6.75e-01	1.44e-01	9.95e-02	3.79e-02	3.05e-05	inf
4	1.26e-15	6.75e-01	1.44e-01	9.96e-02	3.79e-02	7.63e-06	inf
5	0.00e+00	6.75e-0

/datos/MNO_desde_2018/ramas_repo/mno-master/temas/IV.optimizacion_convexa_y_machine_learning/algoritmos/Python/utils_2nd_version.py:55: RuntimeWarning: invalid value encountered in log
  eval_f_const_inequality = np.log(-eval_f_const_inequality)


In [18]:
x

array([1.17048255, 0.84785972, 1.73077939, 0.52009895])

In [19]:
A@x

array([7., 6.])

# Comparación con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [22]:
import cvxpy as cp

In [23]:
x1 = cp.Variable()
x2 = cp.Variable()
x3 = cp.Variable()
x4 = cp.Variable()


In [24]:
# Create two constraints.
constraints = [2*x1+x2+x3+4*x4-7 == 0,x1+x2+2*x3+x4-6 == 0,x1>=0,x2>=0,x3>=0,x4>=0]

# Form objective.

obj = cp.Minimize(x1**2+x2**2+x3**2+x4**2-2*x1-3*x4)

In [25]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.

1.4006849315068515

In [26]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x1.value, x2.value, x3.value,x4.value)

status: optimal
optimal value 1.4006849315068515
optimal var 1.1232876712328763 0.6506849315068494 1.8287671232876717 0.5684931506849316


# Segundo ejemplo

$$\min 2x_1 + 5x_2$$

$$\text{sujeto a: }$$

$$
\begin{array}{c}
6-x_1-x_2 \leq 0 \\
-18 + x_1 +2x_2 \leq 0\\
x_1, x_2 \geq 0
\end{array}
$$

In [20]:
fo = lambda x: 2*x[0] + 5*x[1]

In [21]:
const = {0: lambda x: 6-x[0]-x[1],
         1: lambda x: -18+x[0]+2*x[1],
         2: lambda x: -x[0],
         3: lambda x: -x[1]
        }

In [22]:
A=np.array([0,0],dtype=float)
b = 0

In [23]:
x_ast = np.array([6,0], dtype=float)

In [24]:
#x_0 = np.array([-2,-2], dtype=float)
x_0 = np.array([5,0], dtype=float)
#x_0 = np.array([10,-1], dtype=float)
#x_0 = np.array([6.5,0], dtype=float)

In [25]:
p_ast=fo(x_ast)

In [26]:
p_ast

12.0

In [27]:
nu_0 = np.array([0,0], dtype=float)

In [33]:
tol_outer_iter = 1e-3
tol=1e-8
tol_backtracking=1e-4
maxiter=30
mu=10

In [34]:
fo(x_0)

10.0

In [35]:
x = path_following_method_infeasible_init_point_2nd_version(fo, A, b,
                                                                 const,
                                                               x_0, nu_0, tol,
                                                               tol_backtracking, x_ast, p_ast, maxiter,
                                                               mu, tol_outer_iter = tol_outer_iter 
                                                               )

Outer iterations of path following infeasible method
Mu value: 1.00e+01
LogBarrier 	t_log_barrier	 s_infeasible
4.00e+10	2.00e+00	1.10e+01
----------------------------------------------------------------------------------------
gfeval
[3.87916665 9.89242436 0.        ]
Hfeval
[[0.01556835 0.01347222 0.        ]
 [0.01347222 0.02522001 0.        ]
 [0.         0.         0.        ]]
I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.10e+01	1.53e+01	4.12e+03	1.84e+00	1.67e-01	---		inf
1	0.00e+00	1.08e+01	1.56e+06	8.50e+01	1.73e+02	1.00e+00	inf
2	0.00e+00	1.08e+01	8.17e+11	2.61e+04	6.53e+04	1.00e+00	inf


LinAlgError: Singular matrix

In [36]:
from utils import constraint_inequalities_funcs_eval

In [37]:
constraint_inequalities_funcs_eval(x, const)

array([  3.98165773, -15.13379801,  -1.17048255,  -0.84785972])

# Comparación con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [38]:
x1 = cp.Variable()
x2 = cp.Variable()


In [39]:
# Create two constraints.
constraints = [6-x1-x2 <= 0,-18+x1+2*x2<=0,x1>=0,x2>=0]

# Form objective.

obj = cp.Minimize(2*x1+5*x2)



In [40]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.


12.0000000016275

In [41]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x1.value, x2.value)

status: optimal
optimal value 12.0000000016275
optimal var 6.000000000175689 2.552244387851183e-10


**Referencias:**

* S. P. Boyd, L. Vandenberghe, Convex Optimization, Cambridge University Press, 2009.